In [2]:
import numpy as np
from typing import Any, Union, List, Callable, Dict
from abc import ABC

# Tools

## Parameter

In [3]:
class Parameter:

    def __init__(self, value: float):
        self.value = value

## Referentials

In [56]:
class Referential(ABC):
    @staticmethod
    def from_ref(self, referential: 'Referential') -> np.ndarray:
        pass

class BaseRefential(Referential):
    def from_ref(self, referential: Referential) -> np.ndarray:
        if self == referential:
            return np.eye(4)
        transform_inversed = referential.from_ref(self)
        return np.linalg.inv(transform_inversed)


class Referential(Referential):

    def __init__(self, parent: Referential, params: List[Parameter], transformation: Callable[[float], np.ndarray]):
        self.parent = parent
        self.params = params
        self.transformation = transformation

    def get_transformation_matrix(self):
        return self.transformation(*[param.value for param in self.params])

    def from_ref(self, referential: Referential) -> np.ndarray:
        # From itself, the transformation matrix is the identity matrix
        if self == referential:
            return np.eye(4)
        return  self.parent.from_ref(referential) @ self.get_transformation_matrix()


## Vector

In [79]:
class Vector:

    def __init__(self, x: float, y: float, z: float, referential: Referential):
        self.x = x
        self.y = y
        self.z = z
        self.referential = referential

    def np3(self) -> np.ndarray:
        return np.array([self.x, self.y, self.z])
    
    def np4(self) -> np.ndarray:
        return np.array([self.x, self.y, self.z, 1])

    def in_ref(self, referential: Referential) -> 'Vector':
        np4_in_ref = self.referential.from_ref(referential) @ self.np4()
        return Vector(np4_in_ref[0], np4_in_ref[1], np4_in_ref[2], referential)
    
    def __add__(self, other: Union[np.ndarray, 'Vector']) -> 'Vector':
        if isinstance(other, np.ndarray):
            if not other.shape == (3,):
                raise ValueError("Array must have shape (3,)")
            return Vector(self.x + other[0], self.y + other[1], self.z + other[2], self.referential)
        elif isinstance(other, Vector):
            if self.referential != other.referential:
                raise ValueError("Cannot add Vectors in different referentials")
            return Vector(self.x + other.x, self.y + other.y, self.z + other.z, self.referential)
        else:
            raise ValueError(f"Unsupported type for addition: {type(other)}")
        
    def __sub__(self, other: Union[np.ndarray, 'Vector']) -> 'Vector':
        if isinstance(other, Vector):
            if self.referential != other.referential:
                raise ValueError("Cannot subtract Vectors in different referentials")
            return Vector(self.x - other.x, self.y - other.y, self.z - other.z, self.referential)
        else:
            return self.__add__(-other)
        
    def __mul__(self, scalar: float) -> 'Vector':
        return Vector(self.x * scalar, self.y * scalar, self.z * scalar, self.referential)
    
    def distance(self, other: 'Vector') -> float:
        if self.referential != other.referential:
            raise ValueError("Cannot compute distance between Vectors in different referentials")
        return np.linalg.norm(np.array([self.x, self.y, self.z]) - np.array([other.x, other.y, other.z]))
    
    def __repr__(self):
        return f"Vector({self.x}, {self.y}, {self.z}, {self.referential})"

## A few transformation matrices

In [83]:
def get_body_transormation_matrx(phi: float, psi: float, xi: float, base: np.ndarray) -> np.ndarray:
    cphi, sphi = np.cos(phi), np.sin(phi)
    cpsi, spsi = np.cos(psi), np.sin(psi)
    cxi, sxi = np.cos(xi), np.sin(xi)
    [xbase, ybase, zbase] = base
    return np.array([
        [cphi * cpsi - sphi  * spsi * sxi, -sphi * cxi, cphi * spsi + sphi * cpsi * sxi, xbase],
        [sphi * cpsi + cphi  * spsi * sxi, +cphi * cxi, sphi * spsi - cphi * cpsi * sxi, ybase],
        [-spsi * cxi, sxi, cpsi * cxi, zbase],
        [0, 0, 0, 1]
    ])

def get_rotation_z_transform_matrix(theta: float, translation: np.ndarray) -> np.ndarray:
    ctheta, stheta = np.cos(theta), np.sin(theta)
    [x, y, z] = translation
    return np.array([
        [ctheta, stheta, 0, x],
        [-stheta, ctheta, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

def get_rotation_y_transform_matrix(theta: float, translation: np.ndarray) -> np.ndarray:
    ctheta, stheta = np.cos(theta), np.sin(theta)
    [x, y, z] = translation
    return np.array([
        [ctheta, 0, stheta, x],
        [0, 1, 0, y],
        [-stheta, 0, ctheta, z],
        [0, 0, 0, 1]
    ])

def get_rotation_x_transform_matrix(theta: float, translation: np.ndarray) -> np.ndarray:
    ctheta, stheta = np.cos(theta), np.sin(theta)
    [x, y, z] = translation
    return np.array([
        [1, 0, 0, x],
        [0, ctheta, -stheta, y],
        [0, stheta, ctheta, z],
        [0, 0, 0, 1]
    ])

def get_translation_transform_matrix(translation: np.ndarray) -> np.ndarray:
    [x, y, z] = translation
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

## Test everything put together

In [84]:
params = {
    "phi": Parameter(0),
    "psi": Parameter(0),
    "xi": Parameter(0),
    "dum1": Parameter(0),
    "dum2": Parameter(0),
}

base = BaseRefential()
body = Referential(
    parent=base,
    params=[params["phi"], params["psi"], params["xi"]],
    transformation=lambda phi, psi, xi: get_body_transormation_matrx(phi, psi, xi, np.array([0, 0, 0]))
)
dummy1 = Referential(
    parent=body,
    params=[params["dum1"]],
    transformation=lambda dum1: get_rotation_z_transform_matrix(dum1, np.array([0, 0, 0]))
)
dummy2 = Referential(
    parent=base,
    params=[params["dum2"]],
    transformation=lambda dum2: get_rotation_z_transform_matrix(dum2, np.array([0, 0, 0]))
)

In [85]:
params["phi"].value = 0.
params["psi"].value = 0.
params["xi"].value = 0.
params["dum1"].value = np.pi / 4
params["dum2"].value = np.pi / 4
dummy1.from_ref(dummy2)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

# Leg

In [88]:
class Leg:

    def __init__(self, leg_start_pos: Vector, body: Referential, lengths: List[float]):
        self.leg_start_pos = leg_start_pos
        self.body = body
        self.lengths = lengths
        self._build_leg()

    def _build_leg(self):
        self.parameters = {"alpha": Parameter(0), "beta": Parameter(0), "gamma": Parameter(0)}
        [l1, l2, l3] = self.lengths
        self.referentials = [
            Referential(
                parent=self.body,
                params=[self.parameters["alpha"]],
                transformation=lambda alpha: get_rotation_z_transform_matrix(alpha, self.leg_start_pos.np3())
            ),
        ]
        self.referentials += [
            Referential(
                parent=self.referentials[-1],
                params=[self.parameters["beta"]],
                transformation=lambda beta: get_rotation_y_transform_matrix(beta, np.array([l1, 0, 0]))
            ),
        ]
        self.referentials += [
            Referential(
                parent=self.referentials[-1],
                params=[self.parameters["gamma"]],
                transformation=lambda gamma: get_rotation_y_transform_matrix(gamma, np.array([l2, 0, 0]))
            )
        ]
        self.referentials += [
            Referential(
                parent=self.referentials[-1],
                params=[],
                transformation=lambda: get_translation_transform_matrix(np.array([l3, 0, 0]))
            )
        ]

    def set_angles(self, alpha: float, beta: float, gamma: float):
        self.parameters["alpha"].value = alpha
        self.parameters["beta"].value = beta
        self.parameters["gamma"].value = gamma

    def set_alpha(self, alpha: float):
        self.parameters["alpha"].value = alpha

    def set_beta(self, beta: float):
        self.parameters["beta"].value = beta

    def set_gamma(self, gamma: float):
        self.parameters["gamma"].value = gamma

    def get_angles(self) -> Dict[str, float]:
        return {name: param.value for name, param in self.parameters.items()}
    
    def get_alpha(self) -> float:
        return self.parameters["alpha"].value
    
    def get_beta(self) -> float:
        return self.parameters["beta"].value
    
    def get_gamma(self) -> float:
        return self.parameters["gamma"].value
    
    def get_end_pos(self) -> Vector:
        return self.end_pos
    
    def get_positions(self) -> List[Vector]:
        return [Vector(0, 0, 0, ref) for ref in self.referentials]

## Test leg

In [96]:
leg = Leg(Vector(1, 0, 0, body), body, [2, 3, 4])
leg.set_angles(0, np.pi/4, 0)
[pos.in_ref(body).np3() for pos in leg.get_positions()]

[array([1., 0., 0.]),
 array([3., 0., 0.]),
 array([ 5.12132034,  0.        , -2.12132034]),
 array([ 7.94974747,  0.        , -4.94974747]),
 array([10.77817459,  0.        , -7.77817459])]